# Tracing via the REST API

It's likely that your production LLM application is written in a language other than Python or JavaScript. In this case, you can use the REST API to log runs and take advantage of LangSmith's tracing and monitoring functionality. Doing so can be as simple as a POST request to the langsmith API. Specifically, to log runs you must:

- Submit a POST request to "https://api.smith.langchain.com/runs"
- JSON body of the request must have a name, run_type, inputs, and any other desired information.
- An "x-api-key" header must be provided to authenticate, using a valid API key created in the LangSmith app


## Example

Below is a minimal example of how to create a run using the REST API.

In [8]:
import os
import requests

_LANGSMITH_API_KEY=os.environ["LANGCHAIN_API_KEY"]

res = requests.post(
    "https://api.smith.langchain.com/runs",
    json={
        "name": "MyRun",
        "run_type": "chain",
        "inputs": {"text": "Foo"},
    },
    headers={
        "x-api-key": _LANGSMITH_API_KEY
    }
)

<Response [200]>

This will create a barebones "chain" run with the name "MyRun" and json inputs of {"text": "Foo"} that looks something like the following:

<a href="https://smith.langchain.com/public/956e641a-4f1a-492d-9a67-3990f3fdba3e/r"><img src="./img/minimal.png" alt="minimal trace example" style="width:75%"></a>

Not much information is included, since we haven't added outputs, tags, or metadata yet. It is also marked as "pending" since we haven't added an end time yet.

In addition to the name, run_type, and inputs, you can also provide additional information. Below is an example of the supported schema:

```
{
  "name": "string",
  "inputs": {},
  "run_type": "string",
  "start_time": "2019-08-24T14:15:22Z", # UTC timestamp in ISO format
  "end_time": "2019-08-24T14:15:22Z", # UTC timestamp in ISO format
  "extra": {},
  "error": "string",
  "execution_order": 1,
  "outputs": {},
  "parent_run_id": "f8faf8c1-9778-49a4-9004-628cdb0047e5",
  "events": [
    {}
  ],
  "tags": [
    "string"
  ],
  "id": "497f6eca-6276-4993-bfeb-53cbbbba6f08",
  "session_id": "1ffd059c-17ea-40a8-8aef-70fd0307db82",
  "session_name": "string", # This is the name of the PROJECT. "default" if not specified. Sessions are the old name for projects.
  "reference_example_id": "9fb06aaa-105f-4c87-845f-47d62ffd7ee6"
}
```

This can also be found in the [API documentation](https://web.smith.langchain.com/redoc#tag/run/operation/create_run_runs_post).

Lets look at a more complex chain example:

In [5]:
import uuid
import datetime
import platform

start = datetime.datetime.utcnow() - datetime.timedelta(seconds=10)
end = datetime.datetime.utcnow()
res = requests.post(
    "https://api.smith.langchain.com/runs",
    json={
        "name": "ParentRun",
        "run_type": "chain",
        "inputs": {"text": "Foo"},
        "outputs": {"generated": "Bar"},
        "start_time": start.isoformat(),
        "end_time": end.isoformat(),
        "session_name": "My REST Project",
        "tags": ["langsmith", "rest", "my-example"],
        "events": [
            {"event_name": "retry", "time": start.isoformat()},
            {"event_name": "new_token", "value": "foo"},
        ],
        "extra": {
            "metadata": {
                "my_key": "My value"
            },
            "runtime": {
                "platform": platform.platform(),
            }
        }
    },
    headers={
        "x-api-key": _LANGSMITH_API_KEY
    }
)

Below is an example screenshot of what the logged trace above looks like. The new run now has inputs and outputs, a latency calculation, tags, and a status indicator.

<a href="https://smith.langchain.com/public/69ca776a-60e4-4c4c-8b59-c8731e7f52f0/r"><img src="./img/populated.png" alt="populated trace example" style="width:75%"></a>

To see the logged metadata and other runtime information you saved in the trace above, you can navigate to the "metadata" tab:
    
<a href="https://smith.langchain.com/public/69ca776a-60e4-4c4c-8b59-c8731e7f52f0/r?tab=2"><img src="./img/populated_metadata.png" alt="trace metadata" style="width:75%"></a>

## Logging LLM Runs

The chain runs logged above are versatile and can be used to represent just about any function in your application. To get even more out of LangSmith, you'll want to also log your language model runs.

Runs with the `run_type` of "llm" get some special treatment. They allow us to:

- Help track token usage
- Render "prettier" chat message formats for better readability.

This is the case both for chat models and for regular "completion" LLMs.

The easiest way to log LLM runs to LangSmith is by using OpenAI's llm message schema in the inputs and outputs.

We will show examples below.

#### Logging LLM Chat Messages

To log messages in the "chat" model format (role and message dictionaries), LangSmith expects the following format:

- Provide `messages: [{"role": string, "content": string}]` as a key-value pair in the inputs
- Provide `choices: [{"message": {"role": string, "content": string}]` as a key-value pair in the outputs.

For function calling, you can also pass a `functions=[...]` key-value pair in the inputs, and include a `function_call: {"name": string, "arguments": {}}` key-value pair in the message choice.

In [26]:
requests.post(
    "https://api.smith.langchain.com/runs",
    json={
        "name": "MyChatModelRun",
        "run_type": "llm",
        "inputs": {
            "messages": [{"role": "user", "content": "What's the weather in SF like?"}],
            # Optional
            "model": "text-davinci-003", 
            "functions": [{
              "name": "get_current_weather",
              "description": "Get the current weather in a given location",
              "parameters": {
                "type": "object",
                "properties": {
                  "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                  },
                  "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                  }
                },
                "required": ["location"]
              }}],
            # You can add other invocation paramers as k-v pairs
            "temperature": 0.0,
        },
        "outputs": {
              "choices": [
                    {
                      "index": 0,
                      "message": {
                        "role": "assistant",
                        # Content is whatever string response the
                        # model generates
                        "content": "Mostly cloudy.",
                        # Function call is the function invocation and arguments
                        # as a string
                        "function_call": {
                          "name": "get_current_weather",
                          "arguments": "{\n\"location\": \"San Francisco, CA\"\n}"
                        }
                      },
                      "finish_reason": "function_call"
                    }
                  ],
        },
        "start_time": datetime.datetime.utcnow().isoformat(),
        "end_time": datetime.datetime.utcnow().isoformat(),
    },
    headers={
        "x-api-key": _LANGSMITH_API_KEY
    }
)

<Response [200]>

When viewed in LangSmith, the run will look something like the one below, with the human, AI, and other chat messages all given their own cards, and with the token counts visible on the right.

<a href="https://dev.smith.langchain.com/public/da2d7e1f-c8c7-4cd2-9e42-cbd7b74375bb/r"><img src="./img/chat_example.png" alt="chat example" style="width:75%"></a>


#### Logging "Completions" Models

To log in the "completions" format (string in, string out), LangSmith expects the following format:
- Name the run "openai.Completion.create" or "openai.Completion.acreate"
- Provide `prompt: string` as a key-value pair in the inputs
- Provide `choices: [{"text": string}]` key-value pair in the outputs.

In [29]:
requests.post(
    "https://dev.api.smith.langchain.com/runs",
    json={
        "name": "MyLLMRun",
        "run_type": "llm",
        "inputs": {
            "prompt": "Hi there!",
            # Optional: model or engine name, and other invocation params
            "engine": "text-davinci-003",
            "temperature": 0.0
        },
        "outputs": {
            'choices': [
                {
                   'text': '\nMy name is Polly and I\'m excited to talk to you!',
                   'index': 0,
                   'logprobs': None,
                   'finish_reason': 'stop',
                },
            ]
        },
        "end_time": datetime.datetime.utcnow().isoformat(),
        "start_time": datetime.datetime.utcnow().isoformat(),
        
    },
    headers={
        "x-api-key": os.environ["LANGCHAIN_DEV_API_KEY"] # _LANGSMITH_API_KEY
    }
)

<Response [200]>

The completion model output looks like the example below. Once again, the token counts are indicated on the right, and the completion output is highlighted in green following the origina prompt.

<a href="https://dev.smith.langchain.com/public/de23bfe5-d05e-46c6-aea1-664f159bd50c/r"><img src="./img/completion_example.png" alt="completion example" style="width:75%"></a>

## Updating Runs

The examples above log runs in a single transaction. While simple, this leads to lower responsiveness in the web app and requires you to wait until after an operation has completed before posting any data.

A more responsive approach is to post the inputs and other information when the run starts, then patch the outputs (or error message) when the run completes.

The following example demonstrates this approach. We will first create post/patch functions to handle some of the boilerplate.

In [12]:
import uuid
from typing import Optional

_LANGSMITH_API_KEY=os.environ["LANGCHAIN_API_KEY"]
_LANGSMITH_PROJECT=os.environ.get("LANGCHAIN_PROJECT", "My REST Project")

def post_run(data: dict, name: str, run_id: str) -> None:
    requests.post(
        "https://api.smith.langchain.com/runs",
        json={
            "id": run_id,
            "name": name,
            "run_type": "chain",
            "inputs": data,
            "start_time": datetime.datetime.utcnow().isoformat(),
            "session_name": _LANGSMITH_PROJECT,
        },
        headers={
            "x-api-key": _LANGSMITH_API_KEY
        }
    )

def patch_run(run_id, output: Optional[dict] = None, error: Optional[str] =None) -> None:
    requests.patch(
        f"https://api.smith.langchain.com/runs/{run_id}",
        json={
            "error": error,
            "outputs": output,
            "end_time": datetime.datetime.utcnow().isoformat(),
        },
        headers={
            "x-api-key": _LANGSMITH_API_KEY
        }
    )     

Now we will show how to use these functions in your code:

In [13]:
def my_function(a, b):
    run_id = str(uuid.uuid4())
    post_run({"a": a, "b": b}, "my_function", run_id)
    try:
        result = a + b
        patch_run(run_id, output={"result": result})
    except Exception as e:
        patch_run(run_id, error=str(e))
        raise
    return result   

In [14]:
# Calling the function will always log to LangSmith now.
my_function(1, 2)

3

Below is an example of the logged run. In our case since we are logging in the same thread as the main execution, there is a bit of added latency that is reflected in the run below.

We typically recommend that you perform logging in a background thread.

<a href="https://smith.langchain.com/public/22bcc8e4-208e-4fdc-ae11-ce5329263a88/r"><img src="./img/streamed_result.png" alt="streamed result" style="width:75%"></a>

Now lets try logging with a value that will cause an error.

In [15]:
# This run will raise an error, with the error message logged to LangSmith
try:
    my_function(1, "oops")
except:
    pass

As you can see in the screenshot below, the error has been succesfully logged, and no outputs were returned.

<a href="https://smith.langchain.com/public/d33159dc-d8e7-493c-b5d8-4b6d83e1f401/r"><img src="./img/streamed_error.png" alt="streamed error" style="width:75%"></a>


Before continuing, please be aware of the following requirements for run updates:
- Once an end time has been assigned to a run, it is marked as finished and cannot be updated. 
- Only certain fields can be updated through the patch call (end_time, error, outputs, and events). Other fields can only be set in the initial POST call.

For more information, see the [tracing FAQ](https://docs.smith.langchain.com/tracing/tracing-faq#when-logging-with-the-sdk-which-fields-can-i-update-when-i-patch) in the LangSmith documentation.

## Nesting Runs

The above examples work great for linear logs, but it's likely that your application involves some amount of nested execution. It's a lot easier to debug a complex chain if the logs themselves contain the required associations. There are currently two bits of complexity in doing so. We plan to relax the execution order requirement at some point in the future:

- You must include a `parent_run_id` in your JSON body.
- You must track the `execution_order` of the child run for it to be rendered correctly in the trace.

Below is an updated example of how to do this. We will create a new `RunLogger` class that manages the execution order state for us.

In [16]:
import uuid
from typing import Optional

_LANGSMITH_API_KEY=os.environ["LANGCHAIN_API_KEY"]
_LANGSMITH_PROJECT=os.environ.get("LANGCHAIN_PROJECT", "My REST Project")

class RunLogger:
    def __init__(self):
        self._run_map = {}
        self._parents = {}
        
    def _get_execution_order(self, run_id: str, parent_run_id: Optional[str] = None) -> int:
        if parent_run_id:
            self._parents[run_id] = parent_run_id
            execution_order = self._run_map.get(parent_run_id, 0)
            execution_order += 1
            self._run_map[parent_run_id] = execution_order
        else:
            execution_order = 1
        self._run_map[run_id] = execution_order
        return execution_order

    def _update_execution_order(self, run_id: str) -> None:
        exec_order = self._run_map[run_id]
        parent_run_id = self._parents.pop(run_id, None)
        if parent_run_id:
            self._run_map[parent_run_id] = max(exec_order, self._run_map[parent_run_id])
        
    def post_run(self, data: dict, name: str, run_id: str, parent_run_id: Optional[str] = None) -> None:
        execution_order = self._get_execution_order(run_id, parent_run_id)
        requests.post(
            "https://api.smith.langchain.com/runs",
            json={
                "id": run_id,
                "name": name,
                "run_type": "chain",
                "parent_run_id": parent_run_id,
                "execution_order": execution_order,
                "inputs": data,
                "start_time": datetime.datetime.utcnow().isoformat(),
                "session_name": _LANGSMITH_PROJECT,
            },
            headers={
                "x-api-key": _LANGSMITH_API_KEY
            }
        )

    def patch_run(self, run_id: str, output: Optional[dict] = None, error: Optional[str] =None) -> None:
        self._update_execution_order(run_id)
        requests.patch(
            f"https://api.smith.langchain.com/runs/{run_id}",
            json={
                "error": error,
                "outputs": output,
                "end_time": datetime.datetime.utcnow().isoformat(),
            },
            headers={
                "x-api-key": _LANGSMITH_API_KEY
            }
        )

To demonstrate how this works, we will create a simple fibonacci function and log each call as a "chain" run.

In [17]:
logger = RunLogger()

def fibonacci(n: int, depth: int = 0, parent_run_id: Optional[str] = None) -> int:
    run_id = str(uuid.uuid4())
    logger.post_run({"n": n}, f"fibonacci_recursive", run_id, parent_run_id=parent_run_id)
    try:
        if n <= 1:
            result = n
        else:
            result = fibonacci(n - 1, depth + 1, parent_run_id=run_id) + fibonacci(n - 2, depth + 1, parent_run_id=run_id)
        logger.patch_run(run_id, output={"result": result})
        return result
    except Exception as e:
        logger.patch_run(run_id, error=str(e))
        raise 

In [18]:
fibonacci(5)

5

This should generate a trace similar to the one shown below:
    
<a href="https://smith.langchain.com/public/bd5f59eb-0b42-445c-a57f-352dc985eba4/r"><img src="./img/fibonacci.png" alt="fibonacci" style="width:75%"></a>

All the calls are logged in their correct order.

Similar to before, any error will be logged to LangSmith so you easily see where in the execution the chain failed. Below is an example.

In [22]:
def fibonacci(n: int, depth: int = 0, parent_run_id: Optional[str] = None) -> int:
    run_id = str(uuid.uuid4())
    logger.post_run({"n": n}, f"fibonacci_recursive", run_id, parent_run_id=parent_run_id)
    try:
        if n < 0:
            raise ValueError("NEGATIVE NUMER NOT ALLOWED")
        if n <= 1:
            result = n
        else:
            result = fibonacci(n - 1, depth + 1, parent_run_id=run_id) + fibonacci(n - 2, depth + 1, parent_run_id=run_id)
        logger.patch_run(run_id, output={"result": result})
        return result
    except Exception as e:
        logger.patch_run(run_id, error=str(e))
        raise 
        
# We will show what the trace looks like with an error
try:
    fibonacci(2.3)
except:
    pass

The resulting run should look something like the following. THe errors are propagated up the call hierarchy so you can easily see where in the execution the chain failed.

<a href="https://smith.langchain.com/public/a3065970-1da6-472e-862b-eb9d97fd9607/r"><img src="./img/recursive_error.png" alt="fibonacci with error" style="width:75%"></a>

## Conclusion

In this walkthrough, you used the REST API to log chain and LLM runs to LangSmith and reviewed the resulting traces. This is currently the only way to log runs to LangSmith if you aren't using a language supported by one of the LangSmith SDK's (python and JS/TS). You then used post/patch requests so the results would be usable more quickly and then logged nested runs to take advantage of LangSmith's full trace tree debugging.